# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [15]:
print(df.GarageQual.dtype)

object


In [6]:
# Load necessary packages
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# remove "object"-type features and SalesPrice from `X`

features = [col for col in df.columns if df[col].dtype in [np.float64, np.int64] and col != 'SalePrice']
X = df[features]


# Impute null values
for col in X:
    med = X[col].median()
    X[col].fillna(value = med, inplace  =True)

# Create y
y = df['SalePrice']


Look at the information of `X` again

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [11]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train,  X_test, y_train, y_test = train_test_split(X, y) 

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(linreg.score(X_train, y_train))
print(linreg.score(X_test, y_test))
print(mean_squared_error(y_train, linreg.predict(X_train)))
print(mean_squared_error(y_test, linreg.predict(X_test)))

0.8693272040919765
0.6205968243191029
764892764.340223
2908611554.821565


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [12]:
from sklearn import preprocessing

# Scale the data and perform train test split
transformed = preprocessing.scale(X)
X = pd.DataFrame(transformed, columns = X.columns)

Perform the same linear regression on this data and print out R-squared and MSE.

In [13]:
X_train,  X_test, y_train, y_test = train_test_split(transformed, y) 

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(linreg.score(X_train, y_train))
print(linreg.score(X_test, y_test))
print(mean_squared_error(y_train, linreg.predict(X_train)))
print(mean_squared_error(y_test, linreg.predict(X_test)))

0.8027781699482914
0.8270325634524385
1213204053.4080868
1168048898.2036557


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [16]:
# Create X_cat which contains only the categorical variables

features_cat = [col for col in df.columns if df[col].dtype in [np.object] and col != 'SalePrice']
X_cat = df[features_cat]


# Impute null values
X_cat = pd.get_dummies(X_cat)

In [17]:
# Make dummies
np.shape(X_cat)

(1460, 252)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [18]:
# Your code here

X_all = pd.concat([pd.DataFrame(transformed), X_cat], axis = 1)

Perform the same linear regression on this data and print out R-squared and MSE.

In [20]:
# Your code here

X_train,  X_test, y_train, y_test = train_test_split(X_all, y) 

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(linreg.score(X_train, y_train))
print(linreg.score(X_test, y_test))
print(mean_squared_error(y_train, linreg.predict(X_train)))
print(mean_squared_error(y_test, linreg.predict(X_test)))


0.9062412230820011
-4.540782364610444e+21
570911156.4146118
3.145302420172807e+31


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [23]:
# Your code here
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))
print(mean_squared_error(y_train, lasso.predict(X_train)))
print(mean_squared_error(y_test, lasso.predict(X_test)))

0.9364418120202662
0.8804613250477049
387015272.5100373
828018727.7021382


With a higher regularization parameter (alpha = 10)

In [22]:
# Your code here
lasso = Lasso(alpha=10)
lasso.fit(X_train, y_train)

print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))
print(mean_squared_error(y_train, lasso.predict(X_train)))
print(mean_squared_error(y_test, lasso.predict(X_test)))

0.9350153735133838
0.8887931017846471
395701069.0219812
770306299.6869004


## Ridge

With default parameter (alpha = 1)

In [24]:
# Your code here
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)

print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))
print(mean_squared_error(y_train, ridge.predict(X_train)))
print(mean_squared_error(y_test, ridge.predict(X_test)))

0.9232773249644876
0.8720821323309597
467175794.8171162
886059596.0257934


With default parameter (alpha = 10)

In [25]:
# Your code here
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)

print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))
print(mean_squared_error(y_train, ridge.predict(X_train)))
print(mean_squared_error(y_test, ridge.predict(X_test)))

0.9016850424578816
0.863122307412345
598654418.7997953
948122378.9076169


## Look at the metrics, what are your main conclusions?

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [27]:
# number of Ridge params almost zero
ridge.get_params()

{'alpha': 10,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [28]:
print(sum(abs(ridge.coef_) < 10**(-10)))

2


In [29]:
# number of Lasso params almost zero
print(sum(abs(lasso.coef_) < 10**(-10)))

24


Compare with the total length of the parameter space and draw conclusions!

In [30]:
# your code here
len(lasso.coef_)

289

In [31]:
sum(abs(lasso.coef_) < 10**(-10))/289

0.08304498269896193

## Summary

Great! You now know how to perform Lasso and Ridge regression.